# MLG HW2

In [23]:
import pandas as pd
import numpy as np
import os, time, torch, json
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch_geometric.utils import accuracy,sparse_mx_to_torch_sparse_tensor
import torch_geometric.utils 
# from models.GCN import GCN
import scipy.sparse as sp
from tqdm import tqdm, trange
from torch.autograd import Variable
import torchvision
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torch.utils.data as Data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [37]:
content = []
test = []
train = []
upload = []
edge_index = [] 
for i in range(3):
    os.chdir('/home/rita/111/111-2MLG/HW2/dataset{}'.format(i + 1))
    temp = pd.read_csv('./content.csv', header = None, sep = '\t')
    temp.sort_values(by = [0], inplace = True)
    temp.set_index([0], inplace = True)
    temp = torch.Tensor(np.array(temp)).to(torch.float32)
    content.append(temp)
    test.append(pd.read_csv('./test.csv'))
    temp = pd.read_csv('./train.csv')
    train.append(temp)
    temp = temp[temp.label == 1]
    temp = temp[['to', 'from']]
    temp = temp.reset_index(drop = True)
    edge_index.append(temp)
    upload.append(pd.read_csv('./upload.csv'))
print(content[0][:2])
print(test[0].head(2))
print(train[0].head(2))
print(edge_index[0].head(2))
print(upload[0].head(2))
os.chdir('/home/rita/111/111-2MLG/HW2')

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
       id    to  from
0  E10559  2323  2673
1   E4849    81  1634
       id    to  from  label
0  E10311  2399  2339      0
1  E10255  2397  1144      1
     to  from
0  2397  1144
1  2450  1312
       id  prob
0  E10559   0.5
1   E4849   0.5


In [10]:
class link_predict(nn.Module) :
    def __init__(self, features, emb_dim = 128) :
        super(link_predict, self).__init__()
        self.features = features
        self.edge_index = edge_index
        self.emb_dim = emb_dim
        self.mlp = nn.Sequential(
            nn.Linear(self.features.shape[1], self.emb_dim), 
            nn.ReLU(), 
            nn.Linear(self.emb_dim, self.emb_dim)
        )
        # self.poten_edges = self.get_poten_edges(fearures)
        
    def forward(self, want_edge) :
        z = self.mlp(self.features)
        out = []
        for i in range(want_edge.shape[0]) :
            idx1 = want_edge[i, 0].type(torch.LongTensor)
            idx2 = want_edge[i, 1].type(torch.LongTensor)
            temp = (z[idx1] * z[idx2]).sum()
            # temp = torch.matmul(z[idx1], z[idx2].T)
            temp = temp if temp > 0 else 0
            out.append(temp)
        return torch.tensor(out).reshape(-1, 1).squeeze()

In [11]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.") #可以根據輸出結果知道是否有可用的GPU

Training on device cuda.


In [43]:
def training_loop(model, n_epochs, optimizer, loss_fn, features, train_loader, sigma, val_loader = None) :
    ls_train_loss = []
    ls_val_loss = []
    features = features.to(torch.float32).to(device = device)
    for epoch in range(1, n_epochs + 1) :  
        val_loss = 0
        loop = tqdm(enumerate(train_loader), total = len(train_loader))
        for i, (edge, labels) in loop :
            poten = get_poten_edges(features, edge)
            edge = edge.to(device)
            labels = labels.to(device)
            outputs = model(edge).to(device)            
            loss = 0
            for j in range(labels.shape[0]) :
                if(labels[j] == 1):
                    loss += torch.exp(-(poten[j]/sigma**2)) * loss_fn(outputs[j], labels[j])
                else :
                    loss += torch.exp((poten[j]/sigma**2)) * loss_fn(outputs[j], labels[j])         
            loss.requires_grad_()
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step()
            loop.set_description(f'Epoch[{epoch} / {n_epochs}]')
            # if (i+1 == len(train_loader)) :
            #     val_loss = validate(model, loss_fn, val_loader)
            # loop.set_postfix(loss = loss, val_loss = val_loss)
            loop.set_postfix(loss = loss)
        ls_train_loss.append(loss.item()) 
    
    return ls_train_loss
        
def get_poten_edges(features, edge) :
    poten_edges = []
    for i in range(edge.shape[0]) :
        idx1 = edge[i, 0].type(torch.LongTensor)
        idx2 = edge[i, 1].type(torch.LongTensor)
        temp = (features[idx1] == features[idx2]).sum() 
        poten_edges.append(temp)
    temp -=temp.min()        
    return torch.tensor(poten_edges)    

def validate(model, loss_fn, loader):
    correct = 0
    total = 0
    loss = 0
    with torch.no_grad(): 
        for edges, labels in loader:
            poten = get_poten_edges(features, edges)
            edges = edges.to(device)
            labels = labels.to(device)
            outputs = model(edges).to(device)  
            total += labels.shape[0] 
            for i in range(labels.shape[0]) :
                if(labels[i] == 1):
                    loss += torch.exp(-(poten[i]/sigma**2)) * loss_fn(outputs[i], labels[i])
                else :
                    loss += torch.exp((poten[i]/sigma**2)) * loss_fn(outputs[i], labels[i])     
    loss /= total       
   
    return loss
    
s = time.time()
model = link_predict(content[0].to(device)).to(device)
# L2 regularization
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = 1e-5)  
loss_fn = nn.MSELoss()
n_epochs = 5
features = content[0]
train_x = train[0]
train_y = torch.Tensor(np.array(train_x.iloc[::, -1])).to(torch.float32)
train_x = torch.Tensor(np.array(train_x.iloc[::, [1, 2]])).to(torch.float32)
# train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
train_dataset = Data.TensorDataset(train_x, train_y)
# val_dataset = Data.TensorDataset(val_x, val_y)
batch_size = 64
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 4
)
# val_loader = DataLoader(
#     dataset = val_dataset,
#     batch_size = batch_size,
#     shuffle = True,
#     num_workers = 4
# )
sigma = 100

ls_loss = training_loop( 
    model = model, 
    n_epochs = n_epochs,
    optimizer = optimizer,
    loss_fn = loss_fn,
    features = features, 
    train_loader = train_loader, 
    val_loader = val_loader, 
    sigma = sigma
)
print(time.time() - s)

Epoch[5 / 5]: 100%|██████████| 136/136 [00:03<00:00, 38.51it/s, loss=tensor(12.9162, device='cuda:0', requires_grad=True)]

19.0305073261261


## test

In [44]:
def save(features, train, n_epochs = 100, batch_size = 64, sigma = 100) :
    model = link_predict(features.to(device)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = 1e-5)  
    loss_fn = nn.MSELoss()
    train_y = torch.Tensor(np.array(train.iloc[::, -1])).to(torch.float32)
    train_x = torch.Tensor(np.array(train.iloc[::, [1, 2]])).to(torch.float32)
    # train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
    train_dataset = Data.TensorDataset(train_x, train_y)
    # val_dataset = Data.TensorDataset(val_x, val_y)
    train_loader = DataLoader(
    dataset = train_dataset,
        batch_size = batch_size,
        shuffle = True,
        num_workers = 4
    )
    # val_loader = DataLoader(
    #     dataset = val_dataset,
    #     batch_size = batch_size,
    #     shuffle = True,
    #     num_workers = 4
    # )
    ls_loss = training_loop( 
        model = model, 
        n_epochs = n_epochs,
        optimizer = optimizer,
        loss_fn = loss_fn,
        features = features, 
        train_loader = train_loader, 
        val_loader = val_loader, 
        sigma = sigma
    )
    return ls_loss

loss = {}
for i in range(3) :
    features = content[i]
    train_x = train[i]
    test_x = test[i]
    model = link_predict(features.to(device)).to(device)
    n_epochs = 50
    batch_size = 64
    ls_loss = save(
        features = features, 
        train = train_x, 
        n_epochs = n_epochs, 
        batch_size = batch_size, 
        sigma = 100
    )
    fig = plt.figure()
    plt.title('Loss_{}'.format(i))
    temp = np.array(ls_loss)
    plt.plot(range(1, n_epochs + 1), ls_loss)
    plt.savefig('./figure/loss_{}.png'.format(i + 1))
    plt.close(fig)
    loss[i] = ls_loss
    filename = './model/link_prediction_{}.pt'.format(i + 1)
    torch.save(model.state_dict(), filename)

with open("loss.txt", "w") as fp:
    json.dump(loss, fp)

Epoch[50 / 50]: 100%|██████████| 41/41 [00:01<00:00, 37.10it/s, loss=tensor(2.8152, device='cuda:0', requires_grad=True)] 


In [45]:
def predict(model, test) :
    test_x = torch.Tensor(np.array(test.iloc[::, 1:])).to(device)
    test_y = loaded_model(test_x)
    test_y = pd.DataFrame(test_y)
    pred = pd.concat([test, test_y], axis = 1)
    pred = pred.drop(['to', 'from'], axis = 1)
    pred.columns = ['id', 'prob']
    return pred
for i in range(3) :
    features = content[i]
    test_x = test[i]
    loaded_model = link_predict(features.to(device)).to(device)
    loaded_model.load_state_dict(torch.load('./model/link_prediction_{}.pt'.format(i + 1)))
    loaded_model.to(device)
    pred = predict(
        model = loaded_model, 
        test = test_x
    )
    pred.to_csv('./upload/pred_{}.csv'.format(i + 1))

In [21]:
test_x = torch.Tensor(np.array(test[0].iloc[::, 1:])).to(device)
test_y = loaded_model(test_x)
test_y = pd.DataFrame(test_y)
pred = pd.concat([test[0], test_y], axis = 1)
pred = pred.drop(['to', 'from'], axis = 1)
pred.columns = ['id', 'prob']
pred.to_csv('./upload/pred1.csv')
print(pred)

          id      prob
0     E10559  0.397379
1      E4849  0.400328
2      E3964  0.432398
3       E542  0.377440
4       E331  0.353298
...      ...       ...
2167   E2524  0.345380
2168   E4324  0.421271
2169   E1384  0.419238
2170   E7582  0.369441
2171   E5209  0.362468

[2172 rows x 2 columns]


## 2

In [8]:
s = time.time()
model = link_predict(content[1].to(device)).to(device)
# L2 regularization
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = 1e-5)  
loss_fn = nn.MSELoss()
n_epochs = 50
features = content[1]
train_x = train[1]
train_y = torch.Tensor(np.array(train_x.iloc[::, -1])).to(torch.float32)
train_x = torch.Tensor(np.array(train_x.iloc[::, [1, 2]])).to(torch.float32)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
train_dataset = Data.TensorDataset(train_x, train_y)
val_dataset = Data.TensorDataset(val_x, val_y)
batch_size = 64
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 4
)
val_loader = DataLoader(
    dataset = val_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 4
)
sigma = 100

ls_loss = training_loop( 
    model = model, 
    n_epochs = n_epochs,
    optimizer = optimizer,
    loss_fn = loss_fn,
    features = features, 
    train_loader = train_loader, 
    val_loader = val_loader, 
    sigma = sigma
)
print(time.time() - s)
torch.save(model.state_dict(), './model/d2.pt')

Epoch[50 / 50]: 100%|██████████| 79/79 [00:03<00:00, 22.88it/s, loss=tensor(13.6862, device='cuda:0', requires_grad=True), val_loss=tensor(0.2338, device='cuda:0')]

195.58005237579346


In [10]:
loaded_model = link_predict(content[1].to(device)).to(device)
loaded_model.load_state_dict(torch.load('./model/d2.pt'))
loaded_model.to(device)

link_predict(
  (mlp): Sequential(
    (0): Linear(in_features=3703, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  )
)

In [11]:
test_x = torch.Tensor(np.array(test[1].iloc[::, 1:])).to(device)
test_y = loaded_model(test_x)
test_y = pd.DataFrame(test_y)
pred = pd.concat([test[1], test_y], axis = 1)
pred = pred.drop(['to', 'from'], axis = 1)
pred.columns = ['id', 'prob']
pred.to_csv('./upload/pred2.csv')
print(pred)

         id      prob
0     E3064  0.364130
1      E298  0.342586
2     E3512  0.343670
3     E5670  0.358549
4     E5005  0.379004
...     ...       ...
1881  E9179  0.360409
1882  E5003  0.409637
1883  E5081  0.352319
1884  E4705  0.358406
1885  E1012  0.358096

[1886 rows x 2 columns]


## 3

In [15]:
s = time.time()
model = link_predict(content[2].to(device)).to(device)
# L2 regularization
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = 1e-5)  
loss_fn = nn.MSELoss()
n_epochs = 50
features = content[2]
train_x = train[2]
train_y = torch.Tensor(np.array(train_x.iloc[::, -1])).to(torch.float32)
train_x = torch.Tensor(np.array(train_x.iloc[::, [1, 2]])).to(torch.float32)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
train_dataset = Data.TensorDataset(train_x, train_y)
val_dataset = Data.TensorDataset(val_x, val_y)
batch_size = 64
train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 4
)
val_loader = DataLoader(
    dataset = val_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers = 4
)
sigma = 100

ls_loss = training_loop( 
    model = model, 
    n_epochs = n_epochs,
    optimizer = optimizer,
    loss_fn = loss_fn,
    features = features, 
    train_loader = train_loader, 
    val_loader = val_loader, 
    sigma = sigma
)
print(time.time() - s)
torch.save(model.state_dict(), './model/d3.pt')

Epoch[50 / 50]: 100%|██████████| 27/27 [00:01<00:00, 17.04it/s, loss=tensor(16.1061, device='cuda:0', requires_grad=True), val_loss=tensor(0.2719, device='cuda:0')]

85.13452434539795


In [16]:
loaded_model = link_predict(content[2].to(device)).to(device)
loaded_model.load_state_dict(torch.load('./model/d3.pt'))
loaded_model.to(device)

link_predict(
  (mlp): Sequential(
    (0): Linear(in_features=1703, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  )
)

In [22]:
test_x = torch.Tensor(np.array(test[2].iloc[::, 1:])).to(device)
test_y = loaded_model(test_x)
test_y = pd.DataFrame(test_y)
pred = pd.concat([test[2], test_y], axis = 1)
pred = pred.drop(['to', 'from'], axis = 1)
pred.columns = ['id', 'prob']
pred.to_csv('./upload/pred3.csv')
print(pred)

        id      prob
0     E370  0.362547
1     E667  0.368623
2    E3190  0.410616
3     E848  0.390194
4    E2161  0.383467
..     ...       ...
639   E492  0.422233
640  E3055  0.403786
641  E1271  0.396863
642  E2199  0.351590
643  E2186  0.389092

[644 rows x 2 columns]
